In [ ]:
# import modules, define some functions for loading, saving and processing a gene-barcode matrix
%matplotlib inline
import collections
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp_sparse
import h5py
import csv

np.random.seed(0)

FeatureBCMatrix = collections.namedtuple('FeatureBCMatrix', ['feature_ids', 'feature_names', 'barcodes', 'matrix'])

def get_matrix_from_h5(filename):
    with h5py.File(filename) as f:
        if u'version' in f.attrs:
            if f.attrs['version'] > 2:
                raise ValueError('Matrix HDF5 file format version (%d) is an newer version that is not supported by this function.' % version)
        else:
            raise ValueError('Matrix HDF5 file format version (%d) is an older version that is not supported by this function.' % version)
        
        feature_ids = [x.decode('ascii', 'ignore') for x in f['matrix']['features']['id']]
        feature_names = [x.decode('ascii', 'ignore') for x in f['matrix']['features']['name']]        
        barcodes = list(f['matrix']['barcodes'][:])
        matrix = sp_sparse.csc_matrix((f['matrix']['data'], f['matrix']['indices'], f['matrix']['indptr']), shape=f['matrix']['shape'])
        return FeatureBCMatrix(feature_ids, feature_names, barcodes, matrix)

def get_expression(fbm, gene_name):
    try:
        gene_index = feature_bc_matrix.feature_names.index(gene_name)
    except ValueError:
        raise Exception("%s was not found in list of gene names." % gene_name)
    return fbm.matrix[gene_index, :].toarray().squeeze()

In [ ]:
folder='FLARE2'
project='20190110_Thy1LM'
dataset='Thy1M'

matrices_dir = "/media/storage/ckk/genomics/data/"+folder+"/counts/"+project+"/"+dataset+"/outs/"
save_dir = "/media/storage/ckk/genomics/data/"+folder+"/reanalysis/"+project+"/Barcodes/"
file_matrix_h5 = matrices_dir+"filtered_feature_bc_matrix.h5"
gene_bc_matrix = get_matrix_from_h5(file_matrix_h5)

In [ ]:
# plot distribution of UMIs. Remove cells with < 600 UMIs.
umis_per_cell = np.asarray(gene_bc_matrix.matrix.sum(axis=0)).squeeze()
#plt.hist(np.log10(umis_per_cell), bins=50)
plt.hist((umis_per_cell), bins=50)
plt.xlabel('UMIS per cell')
plt.ylabel('Frequency')
plt.title('UMI Distribution')
plt.show()

del_cells=np.where(umis_per_cell<600)[0]
print("# cells to delete with < 600 UMIs: " + str(len(del_cells)))
print("total # cells to delete: " + str(len(del_cells)))

# plot distrubtion of #genes per cell.
genes_per_cell = np.asarray((gene_bc_matrix.matrix > 0).sum(axis=0)).squeeze()
plt.hist(genes_per_cell, bins=50)
plt.xlabel('Genes per cell')
plt.ylabel('Frequency')
plt.title('Gene Distribution')
plt.show()

# plot distrubtion of #UMIs/#genes per cell. Remove cells with <1.2 ratio
UMIs_to_genes = umis_per_cell/genes_per_cell

plt.hist(UMIs_to_genes, bins=50)
plt.xlabel('UMIs/Genes per cell')
plt.ylabel('Frequency')
plt.title('UMI/Gene Distribution')
plt.show()

print("# cells to delete with UMI/gene ratio < 1.2: " + str(len(np.where(UMIs_to_genes<1.2)[0])))
del_cells = np.append(del_cells, np.where(UMIs_to_genes<1.2)[0])
print("total # cells to delete: " + str(len(del_cells)))

# plot distribution of #cells containing each gene. Remove genes in < 20 cells
cells_per_gene = np.asarray((gene_bc_matrix.matrix>0).sum(axis=1)).squeeze()

plt.hist(cells_per_gene, bins=50)
plt.xlabel('Cells per gene')
plt.ylabel('Frequency')
plt.title('Cells per gene Distribution')
plt.show()

print("# genes to delete in < 20 cells: " + str(len(np.where(cells_per_gene<20)[0])))
del_genes = np.where(cells_per_gene<20)[0]
print("total # genes to delete: " + str(len(del_genes)))

# plot distribution of Mito genes. Remove cells with > 0.3 mito.
gene_series = pd.Series(gene_bc_matrix.feature_names)
mito_genes = gene_series.str.startswith('mt')
mito_genes = np.where(mito_genes==True)[0]
mito_umis_per_cell = np.asarray(gene_bc_matrix.matrix[mito_genes].sum(axis=0)).squeeze()
percent_mito = mito_umis_per_cell/umis_per_cell

plt.hist(percent_mito, bins=50)
plt.xlabel('Percent UMIs coming from mito genes')
plt.ylabel('Frequency')
plt.title('Mito umis Distribution')
plt.show()

print("# cells to delete with > 0.4 mito umis: " + str(len(np.where(percent_mito>0.4)[0])))
del_cells = np.append(del_cells,np.where(percent_mito>0.4)[0])
print("total # cells to delete: " + str(len(del_cells)))

In [ ]:
del_cells=np.unique(del_cells)
qc_barcodes=np.delete(gene_bc_matrix.barcodes,del_cells)
print("total # cells passing QC: " +str(len(qc_barcodes)))

In [ ]:
csvsavefile = save_dir+dataset+'_QC_barcodes.csv'
with open(csvsavefile, 'w', newline='', encoding='utf8') as f: 
    writer = csv.writer(f)
    writer.writerow(["Barcode"])
    for x in range(0,len(qc_barcodes)):
        writer.writerow([qc_barcodes[x].decode('UTF-8')])

In [ ]:
# Exclude sex, activity, or floating RNA genes
exclude_genes=np.array(['Trf','Plp1','Mog','Mobp',"Mfge8","Mbp","Hbb-bs","H2-DMb2","Fos","Jun","Junb","Egr1","Xist","Tsix","Eif2s3y","Uty","Kdm5d"])
for x in range(0,len(exclude_genes)):
    del_genes = np.append(del_genes,np.where(np.asarray(gene_bc_matrix.feature_names) == exclude_genes[x])[0])

In [ ]:
# Exlude mito genes from clustering analysis
del_genes=np.append(del_genes,mito_genes)

# Exclude ribo genes from clustering analysis
ribo_genes = gene_series.str.startswith('Rpl')
ribo_genes = np.where(ribo_genes==True)[0]
del_genes=np.append(del_genes,ribo_genes)
print("total # genes to delete: " + str(len(del_genes)))

In [ ]:
del_genes=np.unique(del_genes)
del_genes_ID=np.asarray(gene_bc_matrix.feature_ids)[del_genes]

csvsavefile = save_dir+dataset+'_QC_excludegenes.csv'
with open(csvsavefile, 'w', newline='', encoding='utf8') as f: 
    writer = csv.writer(f)
    writer.writerow(["Gene"])
    for x in range(0,len(del_genes_ID)):
        writer.writerow([del_genes_ID[x]])

In [ ]:
del_genes_name=np.asarray(gene_bc_matrix.feature_names)[del_genes]

csvsavefile = save_dir+dataset+'_QC_excludegenes_names.csv'
with open(csvsavefile, 'w', newline='', encoding='utf8') as f: 
    writer = csv.writer(f)
    writer.writerow(["Gene"])
    for x in range(0,len(del_genes_name)):
        writer.writerow([del_genes_name[x]])